# Defining a station

A station consists of a Python module with a special function `initialize`. In this function to connect to all your instruments.
For example to use a station one can do:

In [4]:
import qtt.simulation.virtual_dot_array
station = qtt.simulation.virtual_dot_array.initialize()

initialize: create virtualdot
initialized virtual dot system (2 dots)


A station containts Instruments that have Parameter that can be get or set:

In [10]:
gates=station.gates
print(gates)
print(gates.P1.get())
print(station.keithley1)
print(station.keithley1.amplitude.get())

gates: gates (16 gates)
-0.42847433570008486
<VirtualMeter: keithley1>
2.9941489829460344


Typical code included in a station:
* connect to the instruments (the `initialize` function)
* setup parameters
* information about the sample

A sample `initialize` function could be:

<pre>
def initialize(reinit=False, server_name=None):
    global station, mwindows, output_map
    
    logging.info('LD400: initialize')
    print('\n')
    
    if _initialized and not reinit:
        return station
    
    # Loading IVVI
    logging.info('LD400: load IVVI driver')
    ivvi = IVVI.IVVI(name='ivvi', address='COM5', server_name=server_name, numdacs=16,)
    
    # Loading Lockins
    logging.info('LD400: load lockin driver')
    lockin1 = SR830.SR830(name='lockin1', address='GPIB1::6::INSTR', server_name=server_name)
    lockin1.output_interface('GPIB')

    # Loading digitizer
    logging.info('LD400: load digitizer driver')
    digitizer = M4i.M4i(name='digitizer', server_name=server_name)

    logging.info('all drivers have been loaded')
    
    
    # Create map for gates and outputs
    gate_map=station_gate_map()
    gates = virtual_gates(name='gates', gate_map=gate_map, server_name=server_name, instruments=[ivvi])

    #Creating the experimental station
    station = qcodes.Station(ivvi, lockin1, digitizer, gates)
    
    logging.info('Initialized LD400 station')   
    return station
</pre>    

Closing a station is also possible:

In [11]:
qtt.simulation.virtual_dot_array.close()

close gates: gates (16 gates)
close <VirtualMeter: keithley1>
close <VirtualMeter: keithley3>
close <VirtualMeter: keithley4>
close VirtualIVVI: ivvi1
close VirtualIVVI: ivvi2
close <SimulationAWG: vawg>
close <SimulationDigitizer: sdigitizer>
